In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import os
import glob2
import matplotlib.pyplot as plt
import PIL
import pickle
from keras.applications.vgg19 import VGG19
from keras.models import Model
from google.colab import output
from tensorflow.keras.preprocessing import image as image_load
from tensorflow.keras.applications.vgg19 import preprocess_input,decode_predictions
import numpy as np
import string
import time
import pandas as pd
from tqdm import tqdm
from numba import cuda 


In [34]:
def VGG_Model_FCNN():    
    # Defining model
    vgg_19_model = VGG19(weights='imagenet')
    fcnn_layer = Model(inputs=vgg_19_model.layers[0].input, outputs=vgg_19_model.layers[-2].output)
    return fcnn_layer

def image_preparation(img_path):
    final_layer_output = VGG_Model_FCNN()
    # reading image and processing it
    input_image = image_load.load_img(img_path,color_mode='rgb', target_size=(224, 224))
    x = image_load.img_to_array(input_image)
    x = np.expand_dims(x, axis=0)
    processed_image = preprocess_input(x)
    image_features = final_layer_output.predict(processed_image)
    return image_features

def estimateQuestionType(question):
    # print question
    if 'how many' in question:
        typ = 1
    elif question.startswith('what is the color') or \
        question.startswith('what') and 'color' in question:
        typ = 2
    elif question.startswith('where'):
        typ = 3
    else:
        typ = 0
    return typ


def encode_question(question, idx):
    for character in string.punctuation:
        question = question.replace(character, '')
    words = question.lower().split(' ')
    number_of_words = len(words)
    encoded = np.zeros((1, 56, 1), dtype=int)
    encoded[0][0] = idx
    for i in range(number_of_words):
        index = i + 1
        if words[i] == 'colour':
            words[i] = 'color'
        word_encoding = qdict[words[i]]
        encoded[0][index] = word_encoding - 1
    return encoded


def encode_answer(answer):
    for character in string.punctuation:
        answer = answer.replace(character, '')
    word = answer.lower()
    word_encoding = ansdict[word]
    encoded = np.array([word_encoding - 1])
    return encoded

In [35]:
pickle_file_address = '/content/drive/MyDrive/Peter+Galea_stJohn/Python/new_data/'
images_file_address = '/content/drive/MyDrive/SPML-Thesis_Data/Final_Dataset/mydataset_'


qdict = pickle.load(open(pickle_file_address+"qdict.pkl", "rb"), encoding="latin1")
ansdict = pickle.load(open(pickle_file_address+"ansdict.pkl", "rb"))

## **Three Question Types Dataset - Single Word Answer**

In [36]:
classifier_data = pd.read_excel('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/Three Category Dataset/MyCustomDataset_OneWordAnswer_NoDuplicates.xlsx')
classifier_data['img_id'] = classifier_data['img_id'].astype(int)
classifier_data = classifier_data.drop(classifier_data.columns[[0]], axis=1)

In [37]:
classifier_data

,img_id,question,answer
0,1653556733,how many books are on the floor?,four
1,1653556744,how many books are on the floor?,four
2,1653556755,how many books are on the floor?,four
3,1653556766,how many books are under the desk?,four
4,1653556778,what is behind the books?,desk
...,...,...,...
895,1654237143,where are the books?,table
896,1654236949,what is near the bottle?,umbrella
897,1654237740,where is the bottle,couch
898,1654155668,how many knives are on the table?,four


In [38]:
ids = list(classifier_data['img_id'])
len(ids), len(set(ids))

(900, 892)

In [39]:
import tensorflow as tf

# def gpu(images_file_address,vqa_triplet):
  

In [40]:
# # To start
# imgIdToVgg19EncodingListofDictionaries = []
# from_index = 0
imgIdToVgg19EncodingListofDictionaries = pickle.load(open('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/1_Classifier_Completed/Three Question Types/imgIdToVgg19EncodingListofDictionaries_FINAL_NO_DUPLICATES.pkl', "rb"))
from_index = len(imgIdToVgg19EncodingListofDictionaries)
to_index = from_index + 20
print(from_index, to_index)

for imgId in ids[from_index:to_index]:
  vqa_img = images_file_address+str(imgId)+'.jpg'
  new_image = image_preparation(vqa_img)
  imgIdToVgg19EncodingListofDictionaries.append({str(imgId): new_image})
pickle.dump(imgIdToVgg19EncodingListofDictionaries, open('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/1_Classifier_Completed/Three Question Types/imgIdToVgg19EncodingListofDictionaries_FINAL_NO_DUPLICATES.pkl', "wb"))
print('Done')

900 920
Done


imgIdToVgg19EncodingListofDictionaries

In [41]:
imgIdToVgg19EncodingDictionaries = {}
for item in imgIdToVgg19EncodingListofDictionaries:
  imgIdToVgg19EncodingDictionaries[list(item.keys())[0]] = list(item.values())[0]

In [42]:
classifier_data_list = classifier_data.to_dict('records')


In [43]:
list_of_errors = []
x = len(classifier_data_list)
for idx, vqa_triplet in enumerate(classifier_data_list):
  
    question = vqa_triplet['question']
    answer = vqa_triplet['answer']
    q_type = estimateQuestionType(question)
    try:
      type_of_qst = estimateQuestionType(question.lower())
      qst_array = encode_question(question, idx)
      ans_array = encode_answer(answer)
    except:
        if 'handbags' in question:
          amended_question = question.split('handbags')
          question = amended_question[0] + 'hand bags' + amended_question[-1]
          qst_array = encode_question(question, idx)
        elif 'hairbrushes' in question:
          amended_question = question.split('hairbrushes')
          question = amended_question[0] + 'hair brushes' + amended_question[-1]
          qst_array = encode_question(question, idx)
        elif answer == 'handbag':
          answer = 'bag'
          ans_array = encode_answer(answer)
        else:
          list_of_errors.append(idx)

        
    vqa_triplet['question_type']= q_type
    vqa_triplet['encoded_image']= imgIdToVgg19EncodingDictionaries[str(vqa_triplet['img_id'])]
    vqa_triplet['question_array']= qst_array
    vqa_triplet['answer_array']= ans_array

pickle.dump(classifier_data_list, open('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/1_Classifier_Completed/Three Question Types/Classifier_Dataset_FINAL_COMPLETE.pkl', "wb"), protocol=2)



In [44]:
pd.DataFrame(classifier_data_list).to_excel('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/1_Classifier_Completed/Three Question Types/Classifier_Dataset_FINAL_COMPLETE.xlsx')


## **Four Question Types Dataset - Single Word Answer**

In [45]:
classifier_data = pd.read_excel('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/Three Category Dataset/MyCustomDataset_OneWordAnswer_NoDuplicates.xlsx')
classifier_data['img_id'] = classifier_data['img_id'].astype(int)
classifier_data = classifier_data.drop(classifier_data.columns[[0]], axis=1)

In [46]:
classifier_data

,img_id,question,answer
0,1653556733,how many books are on the floor?,four
1,1653556744,how many books are on the floor?,four
2,1653556755,how many books are on the floor?,four
3,1653556766,how many books are under the desk?,four
4,1653556778,what is behind the books?,desk
...,...,...,...
895,1654237143,where are the books?,table
896,1654236949,what is near the bottle?,umbrella
897,1654237740,where is the bottle,couch
898,1654155668,how many knives are on the table?,four


In [47]:
ids = list(classifier_data['img_id'])
len(ids), len(set(ids))

(900, 892)

In [48]:
import tensorflow as tf

# def gpu(images_file_address,vqa_triplet):
  

In [49]:
# # To start
# imgIdToVgg19EncodingListofDictionaries = []
# from_index = 0
imgIdToVgg19EncodingListofDictionaries = pickle.load(open('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/1_Classifier_Completed/Four Question Types/imgIdToVgg19EncodingListofDictionaries_FINAL_NO_DUPLICATES.pkl', "rb"))
from_index = len(imgIdToVgg19EncodingListofDictionaries)
to_index = from_index + 20
print(from_index, to_index)

for imgId in ids[from_index:to_index]:
  vqa_img = images_file_address+str(imgId)+'.jpg'
  new_image = image_preparation(vqa_img)
  imgIdToVgg19EncodingListofDictionaries.append({str(imgId): new_image})
pickle.dump(imgIdToVgg19EncodingListofDictionaries, open('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/1_Classifier_Completed/Four Question Types/imgIdToVgg19EncodingListofDictionaries_FINAL_NO_DUPLICATES.pkl', "wb"))
print('Done')

1000 1020
Done


imgIdToVgg19EncodingListofDictionaries

In [50]:
imgIdToVgg19EncodingDictionaries = {}
for item in imgIdToVgg19EncodingListofDictionaries:
  imgIdToVgg19EncodingDictionaries[list(item.keys())[0]] = list(item.values())[0]

In [51]:
classifier_data_list = classifier_data.to_dict('records')


In [52]:
encode_answer('two')

array([38])

In [53]:
list_of_errors = []
x = len(classifier_data_list)
for idx, vqa_triplet in enumerate(classifier_data_list):
  
    question = vqa_triplet['question']
    answer = vqa_triplet['answer']
    q_type = estimateQuestionType(question)
    
    try:
      type_of_qst = estimateQuestionType(question.lower())
      qst_array = encode_question(question, idx)
      ans_array = encode_answer(answer)
    except:
        if 'handbags' in question:
          amended_question = question.split('handbags')
          question = amended_question[0] + 'hand bags' + amended_question[-1]
          qst_array = encode_question(question, idx)
        elif 'hairbrushes' in question:
          amended_question = question.split('hairbrushes')
          question = amended_question[0] + 'hair brushes' + amended_question[-1]
          qst_array = encode_question(question, idx)
        elif answer == 'handbag':
          answer = 'bag'
          ans_array = encode_answer(answer)
        else:
          list_of_errors.append(idx)
    vqa_triplet['question_type']= q_type
    vqa_triplet['encoded_image']= imgIdToVgg19EncodingDictionaries[str(vqa_triplet['img_id'])]
    vqa_triplet['question_array']= qst_array
    vqa_triplet['answer_array']= ans_array
    

pickle.dump(classifier_data_list, open('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/1_Classifier_Completed/Four Question Types/Classifier_Dataset_FINAL_COMPLETE.pkl', "wb"), protocol=2)



In [54]:
pd.DataFrame(classifier_data_list).to_excel('/content/drive/MyDrive/SPML-Thesis_Data/Datasets/1_Classifier_Completed/Four Question Types/Classifier_Dataset_FINAL_COMPLETE.xlsx')
